# Caso 2: Dashboard Financiero y Scraping
### Nombre: Marco Sebastian Ocampo y Javier Hernández
### Fecha: 7 de noviembre del 2023

In [41]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from datetime import datetime
import dash

from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import dash_bootstrap_components as dbc

In [2]:
dfc= pd.read_csv("CENTROA.csv")
dfc.head()

,Country Name,Time,PIB,CRECIMIENTO,IPC,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Belize,2008,1879425113,-1.519916,100.196857,NaN,NaN,NaN
1,Belize,2009,1870923190,-0.452368,99.090373,NaN,NaN,NaN
2,Belize,2010,1894428507,1.256349,100.000000,NaN,NaN,NaN
3,Belize,2011,1892428054,-0.105597,101.602056,NaN,NaN,NaN
4,Belize,2012,1965444570,3.858351,102.923473,NaN,NaN,NaN


In [19]:
#establcer carpeta donde va a guardar el archivo 
options= webdriver.ChromeOptions()
prefs = {"download.default_directory": r"C:\Users\Marco\Documents\Segundo Semestre 2023\Data Science for Finance"}
options.add_experimental_option("prefs",prefs)
#descargar driver, extraerlo y ponerlo en la carpeta seleccionada arriba
#https://googlechromelabs.github.io/chrome-for-testing/

In [20]:
### INFLACION
s=Service(r"C:\Users\Marco\Documents\Segundo Semestre 2023\Data Science for Finance\chromedriver.exe")
driver=webdriver.Chrome(service=s,options=options)
#determinar la pagina de la que queremos hacer el scraping
driver.get("https://www.banguat.gob.gt/es/page/inflacion-total/")

In [21]:
driver.find_element("xpath",
                    "/html/body/div[2]/div[1]/div/section/div/div/div/div/article/div/div[2]/p[2]/a").click()

In [14]:
### DATA DE INFLACION
dfi = pd.read_excel('INFLACION.xlsx')
dfi.head()

,Año,Mes,Inflacion
0,1996,Enero,9.76
1,1996,Febrero,10.83
2,1996,Marzo,11.48
3,1996,Abril,11.95
4,1996,Mayo,11.02


In [24]:
### TASA DE INTERES
s=Service(r"C:\Users\Marco\Documents\Segundo Semestre 2023\Data Science for Finance\chromedriver.exe")
driver=webdriver.Chrome(service=s,options=options)
#determinar la pagina de la que queremos hacer el scraping
driver.get("https://www.banguat.gob.gt/page/activa")

In [25]:
driver.find_element("xpath",
                    "/html/body/div[2]/div[1]/div/section/div/div/div/div/article/div/div[2]/p[2]/a").click()

In [4]:
### DATA DE TASA DE INTERES
dfi = pd.read_excel('TASA DE INTERES.xlsx')
dfi.head()

,Mes/Año,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Enero,0.222,0.2198,0.1616,0.1781,0.2123,0.1992,0.1771,0.1601,0.1400,...,0.1378,0.1360,0.1308,0.1305,0.1307,0.1274,0.1265,0.1234,0.1192,0.1189
1,Febrero,0.226,0.2121,0.1620,0.1871,0.2139,0.1988,0.1750,0.1584,0.1393,...,0.1381,0.1348,0.1307,0.1305,0.1305,0.1277,0.1268,0.1232,0.1192,0.1194
2,Marzo,0.224,0.2081,0.1637,0.1890,0.2128,0.1958,0.1722,0.1561,0.1389,...,0.1380,0.1337,0.1311,0.1308,0.1301,0.1279,0.1264,0.1232,0.1195,0.1193
3,Abril,0.227,0.1997,0.1614,0.1886,0.2144,0.1942,0.1721,0.1549,0.1397,...,0.1381,0.1329,0.1313,0.1308,0.1302,0.1275,0.1262,0.1228,0.1193,0.1193
4,Mayo,0.229,0.1935,0.1621,0.1902,0.2146,0.1923,0.1711,0.1512,0.1393,...,0.1383,0.1321,0.1315,0.1304,0.1298,0.1274,0.1266,0.1225,0.1194,0.1198


In [5]:
app2 = dash.Dash(__name__)

# Define el diseño de la aplicación
app2.layout = html.Div([
    html.H4("Variación de los indicadores de Centro América"),
    html.P("País:"),
    dcc.Checklist(
        id="x-axis",
        options=[
            {'label': 'Guatemala', 'value': 'Guatemala'},
            {'label': 'Belize', 'value': 'Belize'},
            {'label': 'El Salvador', 'value': 'El Salvador'},
            {'label': 'Honduras', 'value': 'Honduras'},
            {'label': 'Nicaragua', 'value': 'Nicaragua'},
            {'label': 'Costa Rica', 'value': 'Costa Rica'},
            {'label': 'Panama', 'value': 'Panama'}
        ],
        value=['Guatemala'],
        inline=True
    ),
    html.P("Indicador:"),
    dcc.RadioItems(
        id="y-axis",
        options=[
            {'label': 'CRECIMIENTO', 'value': 'CRECIMIENTO'},
            {'label': 'PIB', 'value': 'PIB'},
            {'label': 'IPC', 'value': 'IPC'}
        ],
        value='PIB',
        inline=True
    ),
    dcc.Graph(id="graph"),
    html.P("Tipo de gráfico:"),
    dcc.RadioItems(
        id="chart-type",
        options=[
            {'label': 'Boxplot', 'value': 'boxplot'},
            {'label': 'Líneas', 'value': 'lines'},
            {'label': 'Proporción', 'value': 'pie'}
        ],
        value='boxplot',
        inline=True
    ),
    html.Div(id="year-selector-container")  # Contenedor para el selector de año
])

# Callback para actualizar el gráfico y la selección de año
@app2.callback(
    Output("graph", "figure"),
    Output("year-selector-container", "children"),  # Muestra u oculta el selector de año
    Input("x-axis", "value"),
    Input("y-axis", "value"),
    Input("chart-type", "value")
)
def generate_chart(selected_countries, selected_indicator, chart_type):
    filtered_df = dfc[dfc['Country Name'].isin(selected_countries)]

    if chart_type == 'boxplot':
        # Personalizar la paleta de colores
        colors = ['#33FF57', '#5733FF', '#33FFFF', '#FF33FF']
        fig = px.box(filtered_df, x='Country Name', y=selected_indicator, title=f'Boxplot de {selected_indicator}',
                     color_discrete_sequence=colors)
        year_selector = []  # No mostrar el selector de año
    elif chart_type == 'lines':
        colors1 = ['#FF5733', '#33FF57', '#5733FF', '#33FFFF', '#FF33FF']
        fig = px.line(filtered_df, x='Time', y=selected_indicator, title=f'Gráfico de Líneas de {selected_indicator}',
                      color_discrete_sequence=colors1)
        year_selector = []  # No mostrar el selector de año
    else:
        colors1 = ['#FF5733', '#33FF57', '#5733FF', '#33FFFF', '#FF33FF']
        fig = px.pie(filtered_df, names='Country Name', values=selected_indicator, title=f'Gráfico de Pastel de {selected_indicator}',
                      color_discrete_sequence=colors1)
        year_selector = dcc.Dropdown(
            id="year-selector",
            options=[
                {'label': str(year), 'value': year}
                for year in range(2000, 2023)  # Rango de años a seleccionar
            ],
            value=2022,  # Año inicial seleccionado
            clearable=False,
        )
    
    return fig, year_selector

# Ejecuta la aplicación
if __name__ == '__main__':
    app2.run_server(debug=True, port=2847)

In [20]:
# Construir el dashboard
app = dash.Dash(__name__)

cuentas = ["Inflacion"]

# Layout del app
app.layout = html.Div([
    html.Div([html.Div([
        # Primer drop down para elegir año.
        html.Div(dcc.Dropdown(
            id="Inflacion_año", value=["2019", "2020", "2021", "2022", "2023"], clearable=False, multi=True,
            options=[{'label': x, 'value': x} for x in sorted(dfi.Año.unique())]
        ), className="six columns", style={"width": "50%"},),

        # Segundo drop down
        html.Div(dcc.Dropdown(
            id="Inflacion_total", value="Inflacion", clearable=False,
            options=[{'label': x, 'value': x} for x in cuentas]
        ), className="six columns"),
    ], className="row"),], className="custom-dropdown"),

    # Gráficas
    html.Div([dcc.Graph(id="graph", figure={}, config={"displayModeBar": True, "displaylogo": False,
                                                       # "modeBarButtonsToRemove":['pan2d','lasso2d',
                                                       #                         'select2d']
                                                       })], style={'width': '1100px'}),
    html.Div([dcc.Graph(id="boxplot", figure={})], style={"width": "1100px"}),

    # Tabla
    html.Div(html.Div(id="table-container"), style={'marginBottom': '15px', 'marginTop': "10px"}),
])

# Callback de la función
@app.callback(
    [Output(component_id="graph", component_property="figure"),
     Output(component_id="boxplot", component_property="figure"),
     Output("table-container", 'children')],
    [Input(component_id="Inflacion_año", component_property="value"),
     Input(component_id="Inflacion_total", component_property="value")]
)
# Definición de la función
def display_value(selected_año, selected_mes):
    if len(selected_año) == 0:
        df2 = dfi[dfi["Año"].isin(["2019", "2020", "2021", "2022", "2023"])]
    else:
        df2 = dfi[dfi["Año"].isin(selected_año)]

    # Gráfica 1
    fig = px.line(df2, color="Año", x="Mes", markers=True, y=selected_mes, width=1000, height=500)

    fig.update_layout(title=f'{selected_mes} de {selected_año}', xaxis_title="Inflacion Mensual")
    fig.update_traces(line=dict(width=2))

    # Gráfica 2
    fig2 = px.box(df2, color="Año", x="Año", y=selected_mes, width=1000, height=500)
    fig2.update_layout(title=f'{selected_mes} de {selected_año}')

    # Modificar DataFrame para poder hacerlo tabla
    df_reshaped = df2.pivot(index='Año', columns='Mes', values=selected_mes)
    df_reshaped_3 = df_reshaped.reset_index()

    # Tabla
    return (fig, fig2, dash_table.DataTable(columns=[{"name": i, "id": i} for i in df_reshaped_3],
                                           data=df_reshaped_3.to_dict("records"),
                                           export_format="csv",
                                           fill_width=True,
                                           style_header={'backgroundColor': 'blue', 'color': 'white'}))

# Setear server y correr
if __name__ == '__main__':
    app.run_server(port=2240)
